## 1.导入依赖以及网络结构

In [114]:
# 加载网络结构
%run AI2Flutter.py

## 2.实例化网络，并设置模型输入形状

In [102]:
# 超参数
num_layers = 2
d_model = 128
dff = 2048
num_heads = 8
dropout_rate = 0.2
input_vocab = 1000  # 每个数字在[0,999]之间，997表示起点，998表示生成终点，999表示节点之间的分隔符
output_vocab = 1000  # 

# 权重保存位置
save_weight_path = "./model_weight/demo_model_1"
save_path = "./model/demo_model_1"

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab=input_vocab,
    output_vocab=output_vocab,
    dropout_rate=dropout_rate)

# 使用996作为填充，故考虑loss时不考虑996
def masked_loss(label, pred):
  mask = label != 996
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss

def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 996

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

# 优化器采用Adam，学习率自定义
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy]
)
# 设置模型输入形状
transformer((tf.keras.layers.Input(shape=(None,)),
             tf.keras.layers.Input(shape=(None,))))
# 网络概览
transformer.summary()

Model: "transformer_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_8 (Encoder)         multiple                  2236928   
                                                                 
 decoder_8 (Decoder)         multiple                  3292416   
                                                                 
 dense_80 (Dense)            multiple                  129000    
                                                                 
Total params: 5,658,344
Trainable params: 5,658,344
Non-trainable params: 0
_________________________________________________________________


## 3.加载已经训练的权重，方便继续训练

In [103]:
# 加载已训练权重
transformer.load_weights(save_weight_path)

## 4.加载数据集训练网络

In [124]:
%run AI2Flutter_demo_data.py
%run node_processor.py

In [65]:
%run AI2Flutter_demo_data.py
# 数据规模
train_seqs_num = 500
validation_seqs_num = 50
# 生成随机数据集
input_data, output_data, output_label = demo_generate_data(train_seqs_num)
vali_input_data, vali_output_data, vali_output_label = demo_generate_data(validation_seqs_num)

# 训练集
train_input = tf.data.Dataset.from_generator(
    lambda: input_data, 
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32)))
train_output = tf.data.Dataset.from_generator(
    lambda: output_data, 
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32)))
train_label = tf.data.Dataset.from_generator(
    lambda: output_label, 
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32)))
train_dataset = tf.data.Dataset.zip(((train_input, train_output), train_label))
# batch设置
train_dataset = train_dataset.padded_batch(1, padding_values=996.0)

# 验证集
vali_input = tf.data.Dataset.from_generator(
    lambda: vali_input_data, 
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32)))
vali_output = tf.data.Dataset.from_generator(
    lambda: vali_output_data, 
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32)))
vali_label = tf.data.Dataset.from_generator(
    lambda: vali_output_label, 
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32)))
vali_dataset = tf.data.Dataset.zip(((vali_input, vali_output), vali_label))
vali_dataset = vali_dataset.padded_batch(1, padding_values=996.0)

In [66]:
# 训练
transformer.fit(
    x=train_dataset,
    epochs=10,
    validation_data=vali_dataset
)

Epoch 1/10
500/500 [==============================] - 7s 15ms/step - loss: 0.4145 - masked_accuracy: 0.9363 - val_loss: 0.1627 - val_masked_accuracy: 0.9737
Epoch 2/10
500/500 [==============================] - 7s 15ms/step - loss: 0.3861 - masked_accuracy: 0.9390 - val_loss: 0.0934 - val_masked_accuracy: 0.9870
Epoch 3/10
500/500 [==============================] - 7s 15ms/step - loss: 0.4011 - masked_accuracy: 0.9387 - val_loss: 0.1043 - val_masked_accuracy: 0.9856
Epoch 4/10
500/500 [==============================] - 7s 15ms/step - loss: 0.3341 - masked_accuracy: 0.9463 - val_loss: 0.1095 - val_masked_accuracy: 0.9870
Epoch 5/10
500/500 [==============================] - 7s 15ms/step - loss: 0.4267 - masked_accuracy: 0.9317 - val_loss: 0.0592 - val_masked_accuracy: 0.9924
Epoch 6/10
500/500 [==============================] - 7s 15ms/step - loss: 0.3506 - masked_accuracy: 0.9450 - val_loss: 0.0789 - val_masked_accuracy: 0.9872
Epoch 7/10
500/500 [==============================] - 7s 1

In [ ]:
# num_layers = 2
# d_model = 128
# dff = 512
# num_heads = 8
# dropout_rate = 0.1
# input_node_dim = 24
# target_node_dim = 24
# Model: "transformer_1"
# _________________________________________________________________
#  Layer (type)                Output Shape              Param #   
# =================================================================
#  encoder_1 (Encoder)         multiple                  1322624   
                                                                 
#  decoder_1 (Decoder)         multiple                  2378112   
                                                                 
#  dense_29 (Dense)            multiple                  3096      
                                                                 
# =================================================================
# Total params: 3,703,832
# Trainable params: 3,703,832
# Non-trainable params: 0
# _________________________________________________________________
# 1. 1000(16) 100轮 loss: 1100->255
# 2. 1000(8) 100轮 loss: 281->233
# 3. 1000(4) 100轮 loss: 277->206
# 4. 2000(2) 100轮 loss: 272->244 在12-36个epoch降不下去，像是batch太少
# 5. 1000(8) 100轮 loss: 265->191 再100轮 loss:190->146 再100轮 loss:147->119
# 6. 1000(2) 10轮 loss: 304->239 又被打乱了
# 7. 1000(16) 400轮 loss: 255->102 期间在100附近震荡很多次
# 8. 100(4) 100轮 loss: 373->27
# 9. 100(4) 100轮 loss: 485->36
# 10. 100(4) 100轮 loss: 458->45
# 11. 500(8) 100轮 loss: 438->117

# 注意，transformer由于有填充，损失函数是不考虑这部分的。
# 其次，序列通过均方来作为损失函数，进行回归是不是有问题，很难收敛，
# 考虑到注意力机制是观察另一个向量，是不是应该用分类，而不是向量的回归，

# 此外，我们要AI做到一是布局方式，二是节点裁剪，这里是不是不需要让AI映射每一个值？

## 5.使用网络预测

In [104]:
def d2c(schema, flutter=[], max_length=100):
    '''
    schema: 设计稿schema, 为一个一维向量
    flutter: 实际Flutter节点输出，为一个一维向量，用于测试翻译的结果
    
    特殊数字：996填充，997开始，998结束，999节点分割符
    '''
    # 组织batch为1结构，作为网络输入输出
    input1 = tf.constant([schema])
    predict = [[997]]
    for i in range(max_length):
        p = transformer((input1, tf.constant(predict)), training=False)
        p = p[:, -1:, :]
        p_id = tf.argmax(p, axis=-1)[0].numpy().tolist()[0]
        predict[0].append(p_id)
        if (p_id == 998):
            break
    predict = predict[0][1:]
    print("schema: ", schema)
    if (len(flutter) > 0):
        print("flutter length: ", len(flutter), "value: ", flutter)
    print("predict length: ", len(predict), "value: ", predict)
    if (len(flutter) > 0):
        print("distance: ", [flutter[i] - predict[i] for i in range(min(len(flutter), len(predict)))])
    return predict

In [105]:
# 随机生成一个进行预测
input1, input2, output = demo_generate_data(1)
d2c(input1[0], output[0])

schema:  [1, 0, 0, 67, 91, 26, 255, 0, 255, 0, 75, 71]
flutter length:  11 value:  [2, 1, 0, 26, 255, 0, 255, 0, 75, 71, 998]
predict length:  11 value:  [2, 1, 0, 26, 255, 0, 255, 0, 75, 71, 998]
distance:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[2, 1, 0, 26, 255, 0, 255, 0, 75, 71, 998]

In [146]:
# 编码解码相关操作
def encode_schema_node(node):
    re = []
    if (node["type"] == "text"):
        re.append(1)
    re.extend([node["ax"], node["ay"], node["width"], node["height"]])
    re.append(text_poll.index(node["text"]))
    re.extend(formatColorStrToInt(node["color"]))
    re.extend([node["size"], node["line"]])
    return re

def decode_flutter_node(vec):
    # padding编码
    re = {}
    if (vec[0] == 1):
        re["type"] = "Padding"
        re["id"] = vec[1]
        re["parentId"] = vec[2]
        re["padding"] = str(vec[3]) + ", " + str(vec[4]) + ", " + str(vec[5]) + ", "  + str(vec[6])
    if (vec[0] == 2):
        re["type"] = "TGText"
        re["id"] = vec[1]
        re["parentId"] = vec[2]
        re["text"] = text_poll[vec[3]]
        re["color"] = formatColorIntToStr(vec[4:8])
        re["size"] = vec[8]
        re["line"] = vec[9]
    return re

def decode_flutter_nodes(vec):
    # 去掉998
    vec = vec[: -1]
    # 前后填充
    vec.insert(0, -1)
    vec.append(-1)
    split = [i for i in range(len(vec)) if vec[i] == 999]
    split.insert(0, 0)
    split.append(len(vec) - 1)
    nodes = [vec[split[i-1]+1 : split[i]] for i in range(1, len(split))]
    nodes[-1].append(vec[-1])
    return [decode_flutter_node(node) for node in nodes]
        
def predict_flutter_nodes(schema_node):
    re = decode_flutter_nodes(d2c(encode_schema_node(schema_node)))
    print("\n\n*******")
    print("schema输入： ")
    print(schema_node)
    print("flutter预测：")
    for node in re:
        print(node)
    return re

## 5.1使用node到node翻译

In [154]:
text_poll = ["你好", "英雄联盟手游", "hello", "ahdafwj"]
text_node = {"type":"Text",
             "ax":16,
             "ay":16,
             "width":120,
             "height":52,
             "text": "英雄联盟手游",
             "color":"778E0000",
             "size":24,
             "line":32}

re = predict_flutter_nodes(text_node)

schema:  [1, 16, 16, 120, 52, 1, 119, 142, 0, 0, 24, 32]
predict length:  19 value:  [1, 1, 0, 16, 16, 0, 0, 999, 2, 2, 1, 1, 119, 142, 0, 0, 24, 32, 998]


*******
schema输入： 
{'type': 'text', 'ax': 16, 'ay': 16, 'width': 120, 'height': 52, 'text': '英雄联盟手游', 'color': '778E0000', 'size': 24, 'line': 32}
flutter预测：
{'type': 'Padding', 'id': 1, 'parentId': 0, 'padding': '16, 16, 0, 0'}
{'type': 'TGText', 'id': 2, 'parentId': 1, 'text': '英雄联盟手游', 'color': '778E0000', 'size': 24, 'line': 32}


## 6.保存模型的权重，方便下一次训练

In [84]:
#保存训练权重
transformer.save_weights(save_weight_path)

## 7.保存整个模型，方便迁移到其他地方

In [85]:
# 直接加载模型
# transformer = tf.saved_model.load("model2")
# 保存模型
tf.saved_model.save(transformer, save_path)

INFO:tensorflow:Assets written to: ./model/model_1/assets


INFO:tensorflow:Assets written to: ./model/model_1/assets


## 测试

In [1]:
%run AI2Flutter_demo_data.py

2023-05-05 17:46:31.906622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
text_pool = ['', '英雄联盟手游', '我爱Python']
imgSrc_pool = ['', 'baidu', 'xinyue']
s1 = {'type': 'Text', 'ax': 16, 'ay': 16, 'width': 120, 'height': 52, 'text': '英雄联盟手游', 'color': '778E0000', 'size': 24, 'line': 32}
s2 = {'type': 'Layer', 'ax': 0, 'ay': 0, 'width': 343, 'height': 120, 'color': 'FF8E8E93', 'radius': '12,12,12,12', 'imgSrc': ''}
s3 = {'type': 'Image', 'ax': 150, 'ay': 16, 'width': 100, 'height': 120, 'imgSrc': 'xinyue'}
nodes = [s1, s2, s3]

for node in nodes:
    en = encode_node(node, "Schema", text_pool, imgSrc_pool)
    print(en)
    de = decode_node(en, "Schema", text_pool, imgSrc_pool)
    print(de)

[1, 16, 16, 120, 52, 1, 119, 142, 0, 0, 24, 32]
{'type': 'Text', 'ax': 16, 'ay': 16, 'width': 120, 'height': 52, 'text': '英雄联盟手游', 'color': '778E0000', 'size': 24, 'line': 32}
[2, 0, 0, 343, 120, 255, 142, 142, 147, 12, 12, 12, 12, 0]
{'type': 'Layer', 'ax': 0, 'ay': 0, 'width': 343, 'height': 120, 'color': 'FF8E8E93', 'radius': '12,12,12,12', 'imgSrc': ''}
[3, 150, 16, 100, 120, 2]
{'type': 'Image', 'ax': 150, 'ay': 16, 'width': 100, 'height': 120, 'imgSrc': 'xinyue'}


In [24]:
decode_node(generate_image_data(1)[2][0][:-1], "Flutter", text_pool, imgSrc_pool)

{'type': 'TGRadiusImage',
 'id': 1,
 'parentId': 0,
 'width': 29,
 'height': 353,
 'radius': '2,6,13,10',
 'imgSrc': 'https://weini.com'}

In [3]:
generate_tgButton_data(1)[2][0][:-1]

[7, 1, 0, 48, 187, 179, 121, 33]